# Import libraries

In [1]:
from dotenv import load_dotenv
import os
import random
from tqdm import tqdm
import io
import soundfile as sf

from huggingface_hub import login

from datasets import Dataset, DatasetDict, Audio, load_dataset

import sys
sys.path.append('..')
from utils.audio_util import resample_audios, trim_silence_with_vad
from utils.file_util import recursive_copy

# Huggingface Login

In [2]:
load_dotenv()
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN")

login(token=HF_TOKEN)

# Load Gigaspeech

In [3]:
dataset_path = 'speechcolab/gigaspeech'
cached_path = '../data/raw/gigaspeech'

gigaspeech = load_dataset(path=dataset_path, name='xs', cache_dir=cached_path)
print(gigaspeech)

README.md:   0%|          | 0.00/14.0k [00:00<?, ?B/s]

c:\Users\Ming\anaconda3\envs\voicecraft\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ming\.cache\huggingface\hub\datasets--speechcolab--gigaspeech. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


gigaspeech.py:   0%|          | 0.00/16.6k [00:00<?, ?B/s]

DatasetDict({
    train: Dataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        num_rows: 9389
    })
    validation: Dataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        num_rows: 6750
    })
    test: Dataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        num_rows: 25619
    })
})


In [5]:
# print samples
print(gigaspeech['train'][0])
print(gigaspeech['train'][0]['audio']['sampling_rate']) 

{'segment_id': 'YOU0000000315_S0000660', 'speaker': 'N/A', 'text': "AS THEY'RE LEAVING <COMMA> CAN KASH PULL ZAHRA ASIDE REALLY QUICKLY <QUESTIONMARK>", 'audio': {'path': '../data/raw/gigaspeech/downloads\\extracted\\38fe4c94b1b1d9e64fdcb1171e0c934e20120f08c7da8e780122cf1a55df4cc7\\xs_chunks_0000/YOU0000000315_S0000660.wav', 'array': array([0.0005188 , 0.00085449, 0.00012207, ..., 0.00125122, 0.00076294,
       0.00036621]), 'sampling_rate': 16000}, 'begin_time': 2941.889892578125, 'end_time': 2945.070068359375, 'audio_id': 'YOU0000000315', 'title': 'Return to Vasselheim | Critical Role: VOX MACHINA | Episode 43', 'url': 'https://www.youtube.com/watch?v=zr2n1fLVasU', 'source': 2, 'category': 24, 'original_full_path': 'audio/youtube/P0004/YOU0000000315.opus'}
16000


# Load TSync2

In [5]:
tsync2_path = '../data/raw/TSync2'

wav_files = sorted(os.listdir(f'{tsync2_path}/wav'))
wrd_ph_files = sorted(os.listdir(f'{tsync2_path}/wrd_ph'))

assert len(wav_files) == len(wrd_ph_files)

len_tsync2 = len(wav_files)
print(f'TSync2 has {len_tsync2} audio files')

TSync2 has 2710 audio files


In [6]:
# shuffle pairs
random.seed(42)
combined = list(zip(wav_files, wrd_ph_files))
random.shuffle(combined)

In [7]:
# split train test val
train_val_test = [0.8, 0.1, 0.1] # adjust these values to change the split
train_size = int(len_tsync2 * train_val_test[0])
val_size = int(len_tsync2 * train_val_test[1])
test_size = len_tsync2 - train_size - val_size

train_files = combined[:train_size]
val_files = combined[train_size:train_size + val_size]
test_files = combined[train_size + val_size:]

print(f'Train: {len(train_files)} Val: {len(val_files)} Test: {len(test_files)}')

Train: 2168 Val: 271 Test: 271


# Preprocess the audio files

In [ ]:
!find "../data/raw/TSync2/wav" -type f -name "*.wav" -exec sh -c 'ffmpeg -i "$1" -c:a pcm_mulaw "${1%.wav}.tmp.wav" && mv "${1%.wav}.tmp.wav" "$1"' _ {} \;

In [10]:
# Create destination directory if it doesn't exist
os.makedirs("../data/converted/TSync2-to-gigaspeech/wav16_silence_trimmed", exist_ok=True)

# Copy all files from wav32 to wav16_silence_trimmed
src_dir = "../data/raw/TSync2/wav"
dst_dir = "../data/converted/TSync2-to-gigaspeech/wav16_silence_trimmed"

recursive_copy(src_dir, dst_dir)

In [11]:
# Resample all files in wav16_silence_trimmed to 16kHz
SAMPLE_RATE = 16000
NUM_RESAMPLE_THREADS = 4

resample_audios(
  input_folders=dst_dir,
  file_ext="wav",
  sample_rate=SAMPLE_RATE,
  n_jobs=NUM_RESAMPLE_THREADS
)

Resampling the audio files...
Found 2710 files...


100%|██████████| 2710/2710 [00:07<00:00, 382.73it/s]

Done !


In [12]:
# Trim silence at the beginning and end of each audio file
trim_silence_with_vad(
  input_folder=dst_dir,
  file_extension="wav",
)

Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /Users/titor/.cache/torch/hub/master.zip


Found 2710 .wav files to process


Processing files:   1%|          | 22/2710 [00:00<01:35, 28.17it/s]

> The file ../data/converted/TSync2-to-gigaspeech/wav16_silence_trimmed/tsync2_noon_99_004.wav probably does not have speech please check it !!


Processing files:   2%|▏         | 49/2710 [00:01<01:27, 30.26it/s]

...

> The file ../data/converted/TSync2-to-gigaspeech/wav16_silence_trimmed/tsync2_noon_99_169.wav probably does not have speech please check it !!


Processing files:  99%|█████████▉| 2680/2710 [01:52<00:01, 29.59it/s]

> The file ../data/converted/TSync2-to-gigaspeech/wav16_silence_trimmed/tsync2_noon_99_237.wav probably does not have speech please check it !!


Processing files: 100%|██████████| 2710/2710 [01:53<00:00, 23.78it/s]


Processing complete

Found 58 files with no speech. List saved to ../data/converted/TSync2-to-gigaspeech/no_speech_files.txt


In [13]:
# Normalize the volume of all audio files to -27dB
!find "../data/converted/TSync2-to-gigaspeech/wav16_silence_trimmed" -type f -name "*.wav" -exec sh -c 'ffmpeg-normalize "$1" -nt rms -t=-27 -o "$1" -ar 16000 -f' _ {} \;

# Format Dataset

In [14]:
def load_data(filepairs):
    features = ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path']
    data = {f: [] for f in features}

    for wav_file, wrd_ph_file in tqdm(filepairs):

        file_name = wav_file.split('.')[0]
        
        segment_id = file_name
        speaker = 'Noon'

        with open(f'{tsync2_path}/wrd_ph/{wrd_ph_file}') as f:
            line = f.readline().strip()
            text = ''.join(line.split('|'))
        with open(f'{dst_dir}/{wav_file}', 'rb') as f:
            audio_bytes = f.read()
            
        file_like_object = io.BytesIO(audio_bytes)
        audio_array, sr = sf.read(file_like_object)

        begin_time = 0.0
        end_time = audio_array.shape[0] / sr
        audio_id = file_name
        title = file_name
        url = 'N/A'
        source = 'TSync2'
        category = 10
        original_full_path = ''

        data['segment_id'].append(segment_id)
        data['speaker'].append(speaker)
        data['text'].append(text)
        data['audio'].append(f'{tsync2_path}/wav/{wav_file}')
        data['begin_time'].append(begin_time)
        data['end_time'].append(end_time)
        data['audio_id'].append(audio_id)
        data['title'].append(title)
        data['url'].append(url)
        data['source'].append(source)
        data['category'].append(category)
        data['original_full_path'].append(original_full_path)

    return Dataset.from_dict(data).cast_column("audio", Audio(sampling_rate=16000))


In [15]:
train_data = load_data(train_files)
val_data = load_data(val_files)
test_data = load_data(test_files)

100%|██████████| 271/271 [00:00<00:00, 1109.82it/s]


In [16]:
# print sample data
print(train_data[0])

{'segment_id': 'tsync2_noon_1_2757', 'speaker': 'Noon', 'text': 'ได้ออกเดินทางจากฐานทัพฟอร์ตดรัมในนิวยอร์กแล้ว', 'audio': {'path': '../data/raw/TSync2/wav/tsync2_noon_1_2757.wav', 'array': array([ 5.01572737e-04,  8.84395908e-04,  7.93166109e-04, ...,
       -1.11445050e-04,  7.06672836e-06,  0.00000000e+00]), 'sampling_rate': 16000}, 'begin_time': 0.0, 'end_time': 3.868, 'audio_id': 'tsync2_noon_1_2757', 'title': 'tsync2_noon_1_2757', 'url': 'N/A', 'source': 'TSync2', 'category': 10, 'original_full_path': ''}


In [17]:
dataset_dict = DatasetDict({'train': train_data, 'validation': val_data, 'test': test_data})
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        num_rows: 2168
    })
    validation: Dataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        num_rows: 271
    })
    test: Dataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        num_rows: 271
    })
})


# Push to Huggingface Hub

In [19]:
dataset_dict.push_to_hub('dubbing-ai/tsync2-to-gigaspeech')

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/1084 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

Map:   0%|          | 0/1084 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/271 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/dubbing-ai/tsync2-to-gigaspeech/commit/6057218339dd68cda81873adb4c0facc613eb75c', commit_message='Upload dataset', commit_description='', oid='6057218339dd68cda81873adb4c0facc613eb75c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/dubbing-ai/tsync2-to-gigaspeech', endpoint='https://huggingface.co', repo_type='dataset', repo_id='dubbing-ai/tsync2-to-gigaspeech'), pr_revision=None, pr_num=None)